In [14]:
import numpy as np
import pandas as pd
from statsbombpy import sb
pd.set_option('display.max_rows', None)
from mplsoccer import Pitch, Sbopen
from mplsoccer import VerticalPitch,Pitch
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx



In [16]:
parser = Sbopen()
df_competitions = parser.competition()
df_competitions.head(30)
df = parser.match(competition_id=7, season_id=27)
df.head()

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,...,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3901178,2016-03-12,2016-03-12 18:00:00,1,1,available,unscheduled,2023-08-05 14:29:35.200486,NaT,30,...,1,Regular Season,204.0,Stade Yves Allainmat - Le Moustoir,78.0,France,78.0,Ruddy Buquet,78.0,France
1,3901260,2016-05-14,2016-05-14 21:00:00,1,0,available,unscheduled,2023-08-03 18:34:31.678988,NaT,38,...,1,Regular Season,204.0,Stade Yves Allainmat - Le Moustoir,78.0,France,92.0,Benoît Millot,78.0,France
2,3829470,2015-09-20,2015-09-20 17:00:00,2,0,available,unscheduled,2023-08-03 12:36:31.634292,NaT,6,...,1,Regular Season,4810.0,Stade Geoffroy-Guichard,78.0,France,92.0,Benoît Millot,78.0,France
3,3901259,2016-05-14,2016-05-14 21:00:00,1,1,available,unscheduled,2023-08-06 10:27:05.199641,NaT,38,...,1,Regular Season,176.0,Stade de l''Aube,78.0,France,79.0,François Letexier,78.0,France
4,3901263,2016-05-14,2016-05-14 21:00:00,1,2,available,unscheduled,2023-08-06 22:42:28.958420,NaT,38,...,1,Regular Season,NaN,NaN,NaN,NaN,83.0,Mikael Lesage,78.0,France


In [ ]:
first_row = df.iloc[0]

# Extract 'away_team_name' and 'home_team_name' values from the first row
away_team = first_row['away_team_name']
home_team = first_row['home_team_name']

# Print the values
print("Away Team:", away_team)
print("Home Team:", home_team)


In [13]:
df, related, freeze, tactics = parser.event(3901178)
sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Uruguay"].iloc[0]["index"]
mask_uruguay = (df.type_name == 'Pass') & (df.team_name == "Uruguay") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
df_pass = df.loc[mask_uruguay, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])


IndexError: single positional indexer is out-of-bounds

In [ ]:
df_pass

In [12]:
scatter_df = pd.DataFrame()
for i, name in enumerate(df_pass["player_name"].unique()):
    passx = df_pass.loc[df_pass["player_name"] == name]["x"].to_numpy()
    recx = df_pass.loc[df_pass["pass_recipient_name"] == name]["end_x"].to_numpy()
    passy = df_pass.loc[df_pass["player_name"] == name]["y"].to_numpy()
    recy = df_pass.loc[df_pass["pass_recipient_name"] == name]["end_y"].to_numpy()
    scatter_df.at[i, "player_name"] = name
    #make sure that x and y location for each circle representing the player is the average of passes and receptions
    scatter_df.at[i, "x"] = np.mean(np.concatenate([passx, recx]))
    scatter_df.at[i, "y"] = np.mean(np.concatenate([passy, recy]))
    #calculate number of passes
    scatter_df.at[i, "no"] = df_pass.loc[df_pass["player_name"] == name].count().iloc[0]

#adjust the size of a circle so that the player who made more passes
scatter_df['marker_size'] = (scatter_df['no'] / scatter_df['no'].max() * 1500)

NameError: name 'df_pass' is not defined

In [ ]:
df_pass["pair_key"] = df_pass.apply(lambda x: "_".join(sorted([x["player_name"], x["pass_recipient_name"]])), axis=1)
lines_df = df_pass.groupby(["pair_key"]).x.count().reset_index()
lines_df.rename({'x':'pass_count'}, axis='columns', inplace=True)
lines_df = lines_df[lines_df['pass_count']>2]

In [ ]:
df_pass

In [ ]:
#Drawing pitch
pitch = Pitch(line_color='grey')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
#Scatter the location on the pitch
pitch.scatter(scatter_df.x, scatter_df.y, s=scatter_df.marker_size, color='skyblue', edgecolors='grey', linewidth=1, alpha=1, ax=ax["pitch"], zorder = 3)
#annotating player name
for i, row in scatter_df.iterrows():
    pitch.annotate(row.player_name, xy=(row.x, row.y), c='black', va='center', ha='center', weight = "bold", size=16, ax=ax["pitch"], zorder = 4)

fig.suptitle("Nodes location - Uruguay", fontsize = 30)
plt.show()

In [ ]:
pitch = Pitch(line_color='grey')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
pitch.scatter(scatter_df.x, scatter_df.y, s=scatter_df.marker_size, color='skyblue', edgecolors='grey', linewidth=1, alpha=1, ax=ax["pitch"], zorder = 3)
for i, row in scatter_df.iterrows():
    pitch.annotate(row.player_name, xy=(row.x, row.y), c='black', va='center', ha='center', weight = "bold", size=16, ax=ax["pitch"], zorder = 4)

for i, row in lines_df.iterrows():
        player1 = row["pair_key"].split("_")[0]
        player2 = row['pair_key'].split("_")[1]
        #take the average location of players to plot a line between them
        player1_x = scatter_df.loc[scatter_df["player_name"] == player1]['x'].iloc[0]
        player1_y = scatter_df.loc[scatter_df["player_name"] == player1]['y'].iloc[0]
        player2_x = scatter_df.loc[scatter_df["player_name"] == player2]['x'].iloc[0]
        player2_y = scatter_df.loc[scatter_df["player_name"] == player2]['y'].iloc[0]
        num_passes = row["pass_count"]
        #adjust the line width so that the more passes, the wider the line
        line_width = (num_passes / lines_df['pass_count'].max() * 10)
        #plot lines on the pitch
        pitch.lines(player1_x, player1_y, player2_x, player2_y,
                        alpha=1, lw=line_width, zorder=2, color="skyblue", ax = ax["pitch"])

fig.suptitle("Uruguay Passing Network against South Korea", fontsize = 30)
plt.show()

In [ ]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G
G = digraph(df_pass)
G.edges

In [ ]:

def draw_graph(G):
    layout = nx.spring_layout(G)
    nx.draw(G, layout, with_labels=True, node_size=800, node_color='skyblue')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, layout, edge_labels=labels)
    edge_widths = [weight for _, _, weight in G.edges(data='weight')]
    nx.draw(G, layout, edgelist=G.edges(), width=edge_widths, edge_color='lightblue', edge_cmap=plt.cm.Blues, edge_vmin=min(edge_widths), edge_vmax=max(edge_widths), alpha=0.9)
    plt.show()
draw_graph(G)


In [ ]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G
def match_list(wc):
    matches = list()
    for i in wc['match_id']:
        matches.append(i)
    return matches

data_mf = pd.DataFrame()

In [ ]:
G = digraph(df_pass)


In [ ]:
nx.betweenness_centrality(G)